# DSG 2020: Regression methods (practice)

## Pierre Tandeo, IMT-Atlantique (pierre.tandeo@imt-atlantique.fr)

In this notebook, we apply different regression methods to a real geophysical problem: the Lorenz-63 system. This system is said to be chaotic, meaning that its evolution is highly depending on the initial condition. Consequently, the evolution of this system is hard to predict.

![L63](https://upload.wikimedia.org/wikipedia/commons/1/13/A_Trajectory_Through_Phase_Space_in_a_Lorenz_Attractor.gif "Lorenz-63")

Here, based on a set of observations, the goal is firstly to identify the Lorenz-63 system, meaning that we want to retrieve the equations of the system using learning tools. The second objective is, from the identified data-driven models, to generate artificial trajectories of the Lorenz-63 system.

This practice is largely inspired by this article: https://www.pnas.org/content/pnas/113/15/3932.full.pdf.

### Import library and adjust python parameters

In [ ]:
# import classical libraries
%matplotlib inline
%pylab inline

# avoid warnings
import warnings
warnings.filterwarnings("ignore")

# figure size
rcParams['figure.figsize'] = (16, 9)

# print only 2 decimals
set_printoptions(precision=2)

### Data generation

We generate data following the 3-dimensional Lorenz-63 system, also called the strange attractor, given by:

${\displaystyle {\begin{aligned}{\frac {\mathrm {d} x_1}{\mathrm {d} t}}&=\sigma (x_2-x_1),\\[6pt]{\frac {\mathrm {d} x_2}{\mathrm {d} t}}&=x_1(\rho -x_3)-x_2,\\[6pt]{\frac {\mathrm {d} x_3}{\mathrm {d} t}}&=x_1 x_2-\beta x_3,\end{aligned}}}$

where the physical parameters are $\left(\sigma=10, \rho=28, \beta=8/3\right)$. We use Runge-Kutta 4-5 to integrate the model, using the *odeint()* Python function. The integration time is $\mathrm{d}t=0.001$ and we generate a sequence of $T=100$ Lorenz times.

In [ ]:
# Lorenz-63 dynamical model
def Lorenz_63(x, dx, sigma, rho, beta):
    dx = zeros((3))
    dx[0] = sigma*(x[1]-x[0])
    dx[1] = x[0]*(rho-x[2])-x[1]
    dx[2] = x[0]*x[1] - beta*x[2]
    return dx

In [ ]:
from scipy.integrate import odeint

# define the parameters
x0 = array([8,0,30]) # initial condition
dt = 0.001 # integration time step
T = 100 # number of Lorenz-63 times
sigma = 10
rho = 28
beta = 8/3

# generate the Lorenz-63 system
x = odeint(Lorenz_63, x0, arange(0.01,T,dt), args=(sigma, rho, beta))
time = arange(0.01, T, dt)

### Visualize data

They are 2 ways of visualizing the Lorenz-63 system. The first is to consider the system as a 3-dimensional time series: we plot each variable as a function of time. The second is the phase-space representation where we plot the relationships between variables in 3D and we track the trajectory along time.

In [ ]:
# time series representation
plot(time, x)
xlabel('Lorenz-63 time', size=20)
legend(['x_1','x_2','x_3'], prop={'size': 20})

In [ ]:
# phase-space representation
from mpl_toolkits.mplot3d import Axes3D
fig = figure()
ax = fig.gca(projection='3d')
ax.plot(x[:,0], x[:,1], x[:,2], 'k')
ax.set_xlabel('$x_1$', size=20);ax.set_ylabel('$x_2$', size=20);ax.set_zlabel('$x_3$', size=20)

### Create the training and validation datasets

Here, we want to statistically emulate the Lorenz-63 system, using a regression formulation such that $Y=f(X)$. The Lorenz-63 system is an Ordinary Differential Equation (ODE), where from an initial information at time $t$, the ODE results correspond to increments of each component of the system between $t$ and $t+\mathrm{d}t$.

**Questions:**
- construct the output $Y$ corresponding to the ODE formulation of the Lorenz-63
- construct the input $X$ with the information of the Lorenz-63 at time $t$ (we suggest to take the 3 dimensions and their products)
- create the training dataset noted (*X_train*, *Y_train*) corresponding to the first 2/3 of time series
- create the validation dataset noted (*X_test*, *Y_test*) corresponding to the last part of the time series

In [ ]:
from numpy.random import normal

# output Y
Y = (x[1:,]-x[:-1,:])/dt

# input X    
X = vstack((x[:-1,0], x[:-1,1], x[:-1,2],\
            x[:-1,0]*x[:-1,0], x[:-1,0]*x[:-1,1], x[:-1,0]*x[:-1,2],\
            x[:-1,1]*x[:-1,1], x[:-1,1]*x[:-1,2], x[:-1,2]*x[:-1,2]\
           )).transpose()

# training set
T_train = int(T/dt*2/3) # size of the training set
X_train = X[0:T_train,:]
Y_train = Y[0:T_train,:]

# add noise to training data
#X_train = X_train + random.normal(0, 0.5, shape(X_train))
#Y_train = Y_train + random.normal(0, 0.5, shape(Y_train))

# validation set
X_test = X[T_train+1:,:]
Y_test = Y[T_train+1:,:]

### Apply the multiple linear regression

The regression $Y=f(X)$ can be simply written as a global linear regression without intercept such that: 
\begin{equation}
Y=\sum_{i=1}^p \beta_i X_i.
\end{equation}

**Questions:**
- estimate the $(\beta_1, \dots, \beta_p)$ using the ordinary least squares
- compare the estimated parameters to the true ones
- from the initial values in *X_test*, generate the predicted trajectories using the linear regressions and compare them to the true ones of *Y_test*

In [ ]:
# import functions
from sklearn.linear_model import LinearRegression

# adjust multiple linear regression (mlr) between X and Y
reg_mlr = LinearRegression(fit_intercept=False)
reg_mlr.fit(X_train, Y_train)

# print the estimated parameters
print('Estimated parameters for x_1: ' + str(around(reg_mlr.coef_[0,:], 1)))
print('Estimated parameters for x_2: ' + str(around(reg_mlr.coef_[1,:], 1)))
print('Estimated parameters for x_3: ' + str(around(reg_mlr.coef_[2,:], 1)))

In [ ]:
# function to transform the input and output of the regression to a L63 coordinate
def XY_to_L63(X, Y, dt):
    
    L63 = Y*dt + X
    
    # return the L63 coordinate
    return L63

# function to transform a L3 coordinate to the input of the regression, assuming that:
# X = [x1, x2, x3, x1x1, x1x2, x1x3, x2x2, x2x3, x3x3]
def L63_to_X(trajectory):
    
    X = vstack((trajectory[0], trajectory[1], trajectory[2],\
        trajectory[0]*trajectory[0], trajectory[0]*trajectory[1], trajectory[0]*trajectory[2],\
        trajectory[1]*trajectory[1], trajectory[1]*trajectory[2], trajectory[2]*trajectory[2]\
        )).transpose()
    
    # return the X vector
    return X

# apply the linear regressions (only the firt time step is kept into account)
Y_mlr = reg_mlr.predict(X_test)

# apply sequentially the linear regressions from the initial value of X_test
traj_true = Y_test*0
traj_mlr = Y_test*0
traj_true[0,:] = XY_to_L63(X_test[0,0:3], Y_test[0,:], dt)
traj_mlr[0,:] = XY_to_L63(X_test[0,0:3], Y_mlr[0,:], dt)
for t in range(1,len(X_test)):
    traj_true[t,:] = XY_to_L63(X_test[t,0:3], Y_test[t,:], dt)
    # apply the linear regressions recursively
    traj_mlr[t,:] = XY_to_L63(traj_mlr[t-1,:], reg_mlr.predict(L63_to_X(traj_mlr[t-1,:])), dt)

In [ ]:
# time series representation
plot(time[T_train+2:], traj_true[:,1], 'r')
plot(time[T_train+2:], traj_mlr[:,1], 'k')
xlabel('Lorenz-63 time', size=20)
ylabel('$x_2$', size=20)
legend(['truth','regression'], prop={'size': 20})

In [ ]:
# phase-space representation
fig = figure()
ax = fig.gca(projection='3d')
ax.plot(traj_true[:,0], traj_true[:,1], traj_true[:,2], 'r')
ax.plot(traj_mlr[:,0], traj_mlr[:,1], traj_mlr[:,2], 'k')
ax.set_xlabel('$x_1$', size=20);ax.set_ylabel('$x_2$', size=20);ax.set_zlabel('$x_3$', size=20)

### Study the sensibility of the regression

The current integration time of the Lorenz-63 is too short. Moreover, the training dataset is "clean", in a sense that the observations are not noisy. Thus, this regression problem is relatively easy. Here, we suggest to study the sensibility of the regression to key parameters.

**Questions:**
- reduce the integration time to $\mathrm{d}t=0.01$, compare the identified models and the generated trajectories
- generate a training dataset with an additive Gaussian noise (mean=0, std=0.5), compare the identified models and the generated trajectories

### Apply the local linear regression

As you have seen previously, the global linear regression can not reproduce realistic trajectories of the L63. We thus suggest to implement the local linear regression to improve the results. The local regressions are build from nearest neighbors. Thus, the size of the training dataset need to be increased.